In [1]:
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.datasets import cifar10
from keras import utils

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
img_width, img_height = 32, 32
batch_size = 128
epochs = 50
num_classes = 10
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [5]:
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [7]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False


In [8]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

In [9]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [10]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


In [11]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [ ]:
# Train the model 
model_final.fit(x_train,y_train,
batch_size=batch_size,
epochs = epochs,
validation_data = (x_test,y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.0521 - acc: 0.2456 - val_loss: 1.4976 - val_acc: 0.4639
Epoch 2/50
50000/50000 [==============================] - 63s 1ms/step - loss: 1.3411 - acc: 0.5246 - val_loss: 1.0518 - val_acc: 0.6275
Epoch 3/50
50000/50000 [==============================] - 63s 1ms/step - loss: 1.0865 - acc: 0.6199 - val_loss: 0.9572 - val_acc: 0.6651
Epoch 4/50
50000/50000 [==============================] - 64s 1ms/step - loss: 0.9666 - acc: 0.6647 - val_loss: 0.8485 - val_acc: 0.7020
Epoch 5/50
50000/50000 [==============================] - 64s 1ms/step - loss: 0.8881 - acc: 0.6946 - val_loss: 0.8231 - val_acc: 0.7164
Epoch 6/50
50000/50000 [==============================] - 65s 1ms/step - loss: 0.8253 - acc: 0.7127 - val_loss: 0.7643 - val_acc: 0.7350
Epoch 7/50
50000/50000 [==============================] - 65s 1ms/step - loss: 0.7822 - acc: 0.7309 - val_loss: 0.7565 - val_acc